<a href="https://colab.research.google.com/github/anjalii-s/Machine-Learning-Forecasting-of-ECB-Monetary-Policy-Transmission-to-Latvian-Banking-Liquidity/blob/Dataset-creation-Datasets-and-Python-file/Dataset_creation_adding_EPU_index_and_ECB_rates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Predictive Modeling of ECB Monetary Policy Transmission to Latvijas Banka's Deposit Facility: A Machine Learning Framework for Central Bank Liquidity Forecasting



# Dataset creation

The dataset used is a combination of 3 datasets:

1) Monetary indicators of the Bank of Latvia data available at https://statdb.bank.lv/lb/Data/208. (2014_Analytical_Accounts_of_the_Central_Bank_10_27_2025 downloaded on 27.10.2025).Downloaded from 2014 onwards.

2) European Central Bank rates available at : https://www.ecb.europa.eu/stats/policy_and_exchange_rates/key_ecb_interest_rates/html/index.en.html (ECb rates dataset)

3) Economic policy uncertainity Indices available at : https://www.policyuncertainty.com/all_country_data.html (All_country_data dataset)


# Merge dataset 1 and 2

To create columns with date column as rows .

In [1]:
import pandas as pd
# STEP 1: Create the exact ECB rate timeline manually
def create_ecb_timeline():
    """Create complete monthly ECB deposit facility rate timeline"""
    months = pd.period_range('2014-01', '2025-09', freq='M')
    rates = []
    for month in months:
        if month < pd.Period('2014-06', 'M'): rates.append(0.00)
        elif month < pd.Period('2014-09', 'M'): rates.append(-0.10)
        elif month < pd.Period('2015-12', 'M'): rates.append(-0.20)
        elif month < pd.Period('2016-03', 'M'): rates.append(-0.30)
        elif month < pd.Period('2019-09', 'M'): rates.append(-0.40)
        elif month < pd.Period('2022-07', 'M'): rates.append(-0.50)
        elif month < pd.Period('2022-09', 'M'): rates.append(0.00)
        elif month < pd.Period('2022-11', 'M'): rates.append(0.75)
        elif month < pd.Period('2023-02', 'M'): rates.append(1.50)
        elif month < pd.Period('2023-03', 'M'): rates.append(2.00)
        elif month < pd.Period('2023-05', 'M'): rates.append(2.50)
        elif month < pd.Period('2023-06', 'M'): rates.append(3.00)
        elif month < pd.Period('2023-08', 'M'): rates.append(3.25)
        elif month < pd.Period('2023-09', 'M'): rates.append(3.50)
        elif month < pd.Period('2023-10', 'M'): rates.append(3.75)
        elif month < pd.Period('2024-06', 'M'): rates.append(4.00)
        elif month < pd.Period('2024-09', 'M'): rates.append(3.75)
        elif month < pd.Period('2024-10', 'M'): rates.append(3.50)
        elif month < pd.Period('2024-11', 'M'): rates.append(3.25)
        elif month < pd.Period('2025-02', 'M'): rates.append(3.00)
        elif month < pd.Period('2025-03', 'M'): rates.append(2.75)
        elif month < pd.Period('2025-04', 'M'): rates.append(2.50)
        elif month < pd.Period('2025-06', 'M'): rates.append(2.25)
        else: rates.append(2.00)
    return pd.DataFrame({'Month': months, 'ECB_Deposit_Rate': rates})

# STEP 2: Load and transform Latvijas Banka data
def load_latvia_data():
    """Load and transform the Latvijas Banka data"""
    df = pd.read_csv('2014_Analytical_Accounts_of_the_Central_Bank_10_27_2025.csv')
    df = df.set_index('Item')
    df_transposed = df.T
    df_transposed.index = pd.to_datetime(df_transposed.index, format='%b-%y')
    df_transposed['Month'] = df_transposed.index.to_period('M')
    return df_transposed

# STEP 3: Merge the datasets
def merge_datasets():
    """Merge ECB rates with Latvijas Banka data"""
    latvia_data = load_latvia_data()
    ecb_data = create_ecb_timeline()
    merged_data = latvia_data.merge(ecb_data, on='Month', how='left')
    merged_data = merged_data.set_index(latvia_data.index)
    merged_data.index.name = 'Date'
    merged_data = merged_data.drop('Month', axis=1)
    return merged_data

# STEP 4: Run the merge and save
final_data = merge_datasets()
final_data.to_csv('final_merged_data.csv')
print("✅ Merged data saved to 'final_merged_data.csv'")

✅ Merged data saved to 'final_merged_data.csv'


In [ ]:
# Examine the EPU data structure
epu_data = pd.read_excel('All_Country_Data.xlsx', sheet_name='EPU')
print("EPU Data Shape:", epu_data.shape)
print("\nEPU Data Columns:")
print(epu_data.columns.tolist())
print("\nEPU Data Sample:")
print(epu_data.head())

We are adding EPU to the Latvijas Banka deposit facility dataset because it captures external shocks and uncertainty that significantly influence banks’ liquidity behavior beyond ECB rate changes. Including EPU improves model robustness and forecasting accuracy by accounting for how policy unpredictability amplifies deposit facility usage during different monetary regimes

Since Latvia-specific EPU data is unavailable, Germany’s EPU index is merged into the dataset as a proxy for euro area uncertainty, reflecting external shocks that influence Latvian banks’ deposit facility usage.Latvian banks and businesses are tightly integrated with euro area financial markets, especially Germany.Many ECB transmission studies use Germany EPU as a benchmark for eurozone-wide uncertainty

In [2]:
import pandas as pd
import numpy as np

# Load the main dataset
df = pd.read_csv('final_merged_data.csv')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])
df['YearMonth'] = df['Date'].dt.to_period('M')

# Load EPU data
epu = pd.read_excel('All_Country_Data.xlsx', sheet_name='EPU')

# Process EPU: make Year and Month numeric
epu['Year'] = pd.to_numeric(epu['Year'], errors='coerce')
epu['Month'] = pd.to_numeric(epu['Month'], errors='coerce')

# Drop invalid
epu = epu.dropna(subset=['Year', 'Month'])

# Format strings without .0
epu['Year_str'] = epu['Year'].astype(int).astype(str)
epu['Month_str'] = epu['Month'].astype(int).astype(str).str.zfill(2)  # Pad month to 2 digits

# Create temp date
epu['temp_date'] = pd.to_datetime(epu['Year_str'] + '-' + epu['Month_str'] + '-01')
epu['YearMonth'] = epu['temp_date'].dt.to_period('M')
epu = epu.drop(['temp_date', 'Year_str', 'Month_str'], axis=1)

# Extract Germany EPU as full float precision (no rounding, keep original decimals)
epu['EPU_Germany'] = pd.to_numeric(epu['Germany'], errors='coerce')
epu = epu[['YearMonth', 'EPU_Germany']].dropna(subset=['EPU_Germany'])

# Merge left on YearMonth
df_merged = df.merge(epu, on='YearMonth', how='left')

# Rename
df_merged.rename(columns={'EPU_Germany': 'EPU_Latvia'}, inplace=True)

# Drop YearMonth, move EPU to last
df_merged = df_merged.drop('YearMonth', axis=1)
cols = [col for col in df_merged.columns if col != 'EPU_Latvia'] + ['EPU_Latvia']
df_merged = df_merged[cols]

# Fill NaNs (preserves float precision)
df_merged['EPU_Latvia'] = df_merged['EPU_Latvia'].ffill().bfill()

# Ensure float dtype
df_merged['EPU_Latvia'] = pd.to_numeric(df_merged['EPU_Latvia'], errors='coerce').astype(float)

# Save
df_merged.to_csv('final_merged_with_epu_latvia.csv', index=False)

# Confirmation
print("New dataset created: final_merged_with_epu_latvia.csv")
print("Shape:", df_merged.shape)
print("EPU_Latvia (full float precision) summary:\n", df_merged['EPU_Latvia'].describe())
print("\nSample (Date, EPU_Latvia):\n", df_merged[['Date', 'EPU_Latvia']].head(10))
print("\nData type of EPU_Latvia:", df_merged['EPU_Latvia'].dtype)


New dataset created: final_merged_with_epu_latvia.csv
Shape: (141, 13)
EPU_Latvia (full float precision) summary:
 count     141.000000
mean      381.876944
std       295.169598
min        70.298325
25%       168.087234
50%       231.356674
75%       572.473572
max      1502.183350
Name: EPU_Latvia, dtype: float64

Sample (Date, EPU_Latvia):
         Date  EPU_Latvia
0 2014-01-01  106.185806
1 2014-02-01  134.598724
2 2014-03-01  130.778351
3 2014-04-01   93.582237
4 2014-05-01  129.218002
5 2014-06-01   70.298325
6 2014-07-01  105.196350
7 2014-08-01  142.967789
8 2014-09-01  177.005020
9 2014-10-01  150.627014

Data type of EPU_Latvia: float64
